# 协程
## 2 用作协程的生成器的基本行为

In [1]:
def simple_cor():
    print('-> coroutine started')
    x = yield
    print('-> coroutine received:', x)
my_cor = simple_cor()
my_cor

<generator object simple_cor at 0x000002126FC5A548>

In [2]:
next(my_cor)

-> coroutine started


In [3]:
my_cor.send(42)

-> coroutine received: 42


StopIteration: 

In [4]:
def sim_co(a):
    print('->Started:a = ', a)
    b = yield a
    print('Received:b = ', b)
    c = yield a+b
    print('Received:c = ', c)

In [5]:
m = sim_co(14)
from inspect import getgeneratorstate
getgeneratorstate(m)

'GEN_CREATED'

In [7]:
next(m)

->Started:a =  14


14

In [8]:
m.send(16)

Received:b =  16


30

In [9]:
m.send(69)

Received:c =  69


StopIteration: 

In [10]:
getgeneratorstate(m)

'GEN_CLOSED'

## 3.示例：使用协程计算移动平均值

In [11]:
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total / count

In [12]:
avg = averager()
next(avg      )

In [13]:
avg.send(10)

10.0

In [14]:
avg.send(15)

12.5

## 4 预激协程的装饰器

In [15]:
from functools import wraps
def coroutine(func):
    @wraps(func)
    def primer(*args, **kwargs):
        gen = func(*args, **kwargs)
        next(gen)
        return gen
    return primer

In [16]:
@coroutine
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total / count

In [17]:
a1 = averager()

In [18]:
a1.send(10)

10.0

## 5 终止协程和异常处理

In [19]:
a2 = averager()
a2.send(10)

10.0

In [20]:
a2.send('2')

TypeError: unsupported operand type(s) for +=: 'float' and 'str'

In [22]:
class DemoException(Exception):
    pass
    
def demo_exc_handling():
    print('->coroutine started')
    while True:
        try:
            x = yield
        except DemoException:
            print(' DemoException handled, countinuing')
        else:
            print('-> coroutine received {!r}'.format(x))
    raise RuntimeError('This line should  never run')

In [23]:
exc = demo_exc_handling()
next(exc)
exc.send(11)

->coroutine started
-> coroutine received 11


In [24]:
exc.throw(DemoException)

 DemoException handled, countinuing


In [25]:
getgeneratorstate(exc)

'GEN_SUSPENDED'

In [27]:
class DemoException(Exception):
    pass
    
def demo_exc_handling():
    print('->coroutine started')
    try:
        while True:
            try:
                x = yield
            except DemoException:
                print(' DemoException handled, countinuing')
            else:
                print('-> coroutine received {!r}'.format(x))
    finally:
        print('->coroutine ending')

## 6.让协程返回值

In [28]:
from collections import namedtuple

Result = namedtuple('Result', 'count average')
@coroutine
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield 
        if term is None:
            break
        total += term
        count += 1
        average = total / count
    return Result(count, average)

In [30]:
avg2 = averager()
avg2.send(11)
avg2.send(12)
avg2.send(15)
"""try:
    avg.send(None)
except StopIteration as exc:
    res = exc.value
res"""

'try:\n    avg.send(None)\nexcept StopIteration as exc:\n    res = exc.value\nres'

In [31]:
avg2.send(None)

StopIteration: Result(count=3, average=12.666666666666666)

In [32]:
avg2 = averager()
avg2.send(11)
avg2.send(12)
avg2.send(15)
try:
    avg2.send(None)
except StopIteration as exc:
    res = exc.value
res

Result(count=3, average=12.666666666666666)

## 7使用yield from